<a href="https://colab.research.google.com/github/Bendrox/DA_etude_bonvivre/blob/main/Explore_L%C3%A9giFrance_API_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Remain - to do list :

- Fetch / request data :
   - Textcid values
   - elementCid values
   - idEliOrAlias values


- Data prep


- Global architecture


- Function definition


- Interface


- RAG ?







----------------------------------------------------  ----------------------------------------------------  ----------------------------------------------------


# 1 - LégiFrance API co

In [ ]:
import getpass
client_id = getpass.getpass("Enter the client_id Key: ")

In [ ]:
client_secret = getpass.getpass("Enter the client_secret Key: ")

## 1.1 - API Get access token

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


In [ ]:
pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session
import json

In [ ]:
#option 2:
import requests


# URL pour obtenir le jeton
token_url = 'https://sandbox-oauth.piste.gouv.fr/api/oauth/token'

# inject cred for token request
token_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'openid'
}

# run the request
response = requests.post(token_url, data=token_data)
response.raise_for_status()  # vérif  erreurs

# récup  jeton
token_info = response.json()
access_token = token_info['access_token']

## 1.2 - Ping/Pong Test

In [ ]:
# Call 1 : Ping/Pong
headers_1=  {
		'accept': 'text/plain',
		'Authorization': 'Bearer ' + access_token
	}

output = requests.get("https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/ping",
                       headers = headers_1).text

display(output)

'pong'

# 2 - API Call

Environ 70 requetes, en 6 "types"/"familles" de requêtes:

1 - Consult Controller: pour consulter des informations spécifiques sur un document législatif ou réglementaire, tel qu un article ou un texte précis.

-> Nécessaire

2 - List Controller: permet d'obtenir des listes de documents selon des critères spécifiques, comme les codes, lois, décrets, etc. Exemple obtenir une liste de textes d un code particulier

-> Peut etre nécessaire.


3 - Suggest Controller: aide à fournir des suggestions basées sur une recherche partielle. Appel pour rechercher un texte par mot-clé

-> Peut etre nécessaire.


4 - Misc Controller: pour récupérer des métadonnées ou des informations auxiliaires.

-> Pas nécessaire

5 - Search Controller fait des recherches globales sur les textes, articles, codes, etc., en fonction de mots-clés ou d'autres critères.
Exemple d'appel pour rechercher un texte par mot-clé.

-> Peut etre nécessaire.


6 - Chrono Controller : pour accéder à l'historique et à l'évolution chronologique des textes.

-> Nécessaire


## 2.1 - Consult Controller

### 2.1.1 - Contenu Article - avec Eli






Eli - Identifiant Européen de la Législation:

- Permet d'identifier de façon unique les documents législatifs et réglementaires au sein de la base de données de Légifrance.

- Déclinaison :

/eli : Préfixe indiquant qu'il s'agit d'un identifiant ELI.

/decret : Type de texte (ici un décret) peut aussi être un arrêté, une loi, ou d'autres types d'actes normatifs...

/2021/7/13 : Date de publication ou adoption (13 juillet 2021 dans cet exemple).

/PRMD2117108D : Numéro unique du décret

/jo : Indication que le document a été publié dans le Journal Officiel.

/article_1 : Référence à un article spécifique dans le document (ici l'article 1).


In [ ]:
# Call 2: Contenu article
headers_2 = {
    "accept": "application/json",
    "Content-Type": "application/json",
    'Authorization': 'Bearer ' + access_token}

data = {
    "idEliOrAlias": "/eli/decret/2021/7/13/PRMD2117108D/jo/article_1"
}

url= "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/consult/getArticleWithIdEliOrAlias"
response = requests.post(url,
                         headers=headers_2,
                         json=data) # dictionnaire data converti en JSON
                                    #inject request

In [ ]:
response_json = response.json()
display(response_json)


In [ ]:
response_json['article']['context']['titresTM']
# input : Eli
# reste a dev : structure (ex: livre , chapitre, section ...)
# dates valeur abérante

[{'debut': '2999-01-01',
  'fin': '2999-01-01',
  'titre': 'Chapitre Ier : Attributions du secrétaire général de la défense et de la sécurité nationale en matière de lutte contre les ingérences numériques étrangères',
  'xPath': '/ARTICLE/CONTEXTE/TEXTE/TM/TITRE_TM',
  'cid': 'JORFSCTA000043788368',
  'id': 'JORFSCTA000043788368',
  'etat': ''}]

In [ ]:
response_json['article']['textTitles'][1]['etat']

'VIGUEUR'

In [ ]:
response_json['article']['etat']

'INITIALE'

In [ ]:
response_json['article']['id']  #JORF

'JORFARTI000043788371'

In [ ]:
response_json['article']['textTitles'][1]["titreLong"] # Ok demande DAJ

"Décret n° 2021-922 du 13 juillet 2021 portant création, auprès du secrétaire général de la défense et de la sécurité nationale, d'un service à compétence nationale dénommé « service de vigilance et de protection contre les ingérences numériques étrangères »"

In [ ]:
response_json['article']['idEliAlias'] # Ok demande DAJ pour N°Article

'/eli/decret/2021/7/13/2021-922/jo/article_1'

In [ ]:
response_json['article']['texte']

"Le code de la défense est ainsi modifié : 1° L'article R.* 1132-3 est complété par un 9° ainsi rédigé : « 9° En liaison avec les départements ministériels concernés, il identifie les opérations impliquant, de manière directe ou indirecte, un Etat étranger ou une entité non étatique étrangère, et visant à la diffusion artificielle ou automatisée, massive et délibérée, par le biais d'un service de communication au public en ligne, d'allégations ou imputations de faits manifestement inexactes ou trompeuses de nature à porter atteinte aux intérêts fondamentaux de la Nation. Il anime et coordonne les travaux interministériels en matière de protection contre ces opérations. » ; 2° La section 1 du chapitre II du titre III du livre premier de la première partie est complétée par un article D. 1132-8 ainsi rédigé : « Art. D. 1132-8. - Pour assurer les missions qui lui sont assignées au 9° de l'article R.* 1132-3, le secrétaire général de la défense et de la sécurité nationale dispose du servic

## 2.6 - Chrono Controller




1 - Extrait d'une version d'un texte:
POST
/chrono/textCidAndElementCid - ok

2 - Vérifie si un texte possède des versions:
GET
/chrono/textCid/{textCid} - ok

3 - Version d'un texte:
POST
/chrono/textCid - ok

4 - Test controller:
GET
/chrono/ping

### 2.6.1  Extrait d'une version d'un texte : chrono/textCidAndElementCid

Récupère un extrait (section ou article) d'une version spécifique d'un texte à partir des identifiants du texte (textCid) et de l'extrait (section ou article: "elementCid")


In [ ]:
import requests

headers = {
		'accept': 'application/json',
		'Content-Type': 'application/json',
		'Authorization': 'Bearer ' + access_token
	}

url = "https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCidAndElementCid"

# elementCid id unique pour contenu juridique :
# exemple  article de loi,  section,  chapitre ou  autre subdivision juridique.

data = {
    "textCid": "LEGITEXT000006072050", # code du travail
    "elementCid": "LEGIARTI000006901817" # Article L2262-8
}
# Test 1: "textCid": "LEGITEXT000006070721" Code Civil, "elementCid": "LEGIARTI000038033084" -> NA
# Test 2: "textCid": "LEGITEXT000006070721" Code Civil , "elementCid": "LEGIARTI000006417324" -> NA
# Test 3 : "textCid": "LEGITEXT000006072050" Code du Travail + Article L1221-1 "elementCid": "LEGIARTI000006901817" -> ok
# https://www.legifrance.gouv.fr/codes/article_lc/LEGIARTI000006900839

In [ ]:
#  requête GET
response_2_6_1 = requests.post(url, headers=headers, json= data)

In [ ]:
display(response_2_6_1.json())
# contenu manquant !

{'executionTime': 0,
 'regroupements': [{'title': '2008',
   'versions': {'2008-05-01': {'dateDebut': 1209600000000,
     'isEndVersion': False,
     'articlesModificateurs': {'JORFTEXT000000465978': {'title': 'Ordonnance 2007-329 2007-03-12 JORF 13 mars 2007',
       'id': 'JORFTEXT000000465978',
       'nature': 'CODE',
       'dateDebutCible': 32472144000000,
       'actions': {'CODIFICATION': {'action': 'CODIFICATION',
         'parents': {'LEGISCTA000006189538': {'name': 'Section 2 : Information et communication.',
           'id': 'LEGISCTA000006189538',
           'cid': 'LEGISCTA000006189538',
           'cidText': 'LEGITEXT000006072050',
           'nature': 'CODE',
           'dateDebut': 1209600000000,
           'articlesCibles': {'LEGIARTI000006901817': {'name': 'L2262-8',
             'id': 'LEGIARTI000006901817',
             'cid': 'LEGIARTI000006901817',
             'idParent': 'LEGISCTA000006189538',
             'cidParent': 'LEGISCTA000006189538',
             'cid

### 2.6.2  Version d'un texte /chrono/textCid (Binary output)



def text_cid:

LEGITEXT000006073984 # code des assurances

LEGITEXT000006070721 # code civil

LEGITEXT000005634379 # Code de commerce

LEGITEXT000006070719 # code pénal  

LEGITEXT000006072026 # cmf

In [ ]:
import requests


text_cid = "LEGITEXT000006073984" # input

# URL de l'API + inject textCid
url = f"https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCid/{text_cid}"


headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + access_token
}

# Envoi  requête GET
response = requests.get(url, headers=headers)
data = response.json()
display(data)


{'executionTime': 2, 'chronolegiExists': True}

In [ ]:
type(data)

dict

In [ ]:
def version_exist(text_cid):
  url = f"https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCid/{text_cid}"
  headers = {
    "accept": "application/json",
        "Authorization": "Bearer " + access_token}
  response = requests.get(url, headers=headers)
  data = response.json()
  return display(data["chronolegiExists"])

In [ ]:
version_exist("LEGITEXT000006073984")
# LEGITEXT000006073984 # code des assurances : oui
# LEGITEXT000006070721 # code civil : oui
# LEGITEXT000005634379 # Code de commerce : oui
# LEGITEXT000006070719 # code pénal  oui
# LEGITEXT000006072026 # cmf

True

### 2.6.3 - Récupère une version (date) d'un texte (code) à partir de l'id textCid - Ok v2 data prep




#### 2.6.3.1 - Requête

Requête prend en entrée réf code et année pour afficher les changements réglementaires.

In [ ]:
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + access_token
}

# Données de la requête
data = {
  "endYear": 2023,
  "dateConsult": "2021-04-15",
  "startYear": 2015,
  "textCid": "LEGITEXT000006070721"
}

url = 'https://sandbox-api.piste.gouv.fr/dila/legifrance/lf-engine-app/chrono/textCid'

# Envoi de la requête POST
response = requests.post(url, headers=headers, json=data)
display(response)

<Response [200]>

In [ ]:
type(response.json)

method

In [ ]:
data_json_2_6_3 = response.json()

In [ ]:
type(data_json_2_6_3)

dict

In [ ]:
response.text

'{"executionTime":48,"regroupements":[{"title":"2026","versions":{"2026-12-31":{"dateDebut":1798675200000,"isEndVersion":false,"articlesModificateurs":{}},"2026-01-01":{"dateDebut":1767225600000,"isEndVersion":false,"articlesModificateurs":{}}},"detailLoaded":false},{"title":"2024","versions":{"2024-09-14":{"dateDebut":1726272000000,"isEndVersion":false,"articlesModificateurs":{}},"2024-07-05":{"dateDebut":1720137600000,"isEndVersion":false,"articlesModificateurs":{}},"2024-06-15":{"dateDebut":1718409600000,"isEndVersion":false,"articlesModificateurs":{}},"2024-06-02":{"dateDebut":1717286400000,"isEndVersion":false,"articlesModificateurs":{}},"2024-04-17":{"dateDebut":1713312000000,"isEndVersion":false,"articlesModificateurs":{}},"2024-03-20":{"dateDebut":1710892800000,"isEndVersion":false,"articlesModificateurs":{}},"2024-02-21":{"dateDebut":1708473600000,"isEndVersion":false,"articlesModificateurs":{}}},"detailLoaded":false},{"title":"2023","versions":{"2023-05-21":{"dateDebut":16846

#### 2.6.3.2 Data structure

In [ ]:
import json

print('Valo 1er niveau: ')
data_json_2_6_3.keys()

Valo 1er niveau: 


dict_keys(['executionTime', 'regroupements', 'datePublication'])

In [ ]:
type(data_json_2_6_3['regroupements'])

list

In [ ]:
type(response.content)

bytes

In [ ]:
response.json().keys()

dict_keys(['executionTime', 'regroupements', 'datePublication'])

In [ ]:
data_json_2_6_3['regroupements'][0]

{'title': '2026',
 'versions': {'2026-12-31': {'dateDebut': 1798675200000,
   'isEndVersion': False,
   'articlesModificateurs': {}},
  '2026-01-01': {'dateDebut': 1767225600000,
   'isEndVersion': False,
   'articlesModificateurs': {}}},
 'detailLoaded': False}

In [ ]:
print("Valo cle 3iem niveau:")
data_json_2_6_3['regroupements'][0].keys()

Valo cle 3iem niveau:


dict_keys(['title', 'versions', 'detailLoaded'])

In [ ]:
type(data_json_2_6_3['regroupements'][0]['versions'])

dict

In [ ]:
print("Valo cle 4iem niveau ")
data_json_2_6_3['regroupements'][0]['versions'].keys() # version_date, version_data

Valo cle 4iem niveau 


dict_keys(['2026-12-31', '2026-01-01'])

In [ ]:
data_json_2_6_3['regroupements'][0]['versions']['2026-12-31']

{'dateDebut': 1798675200000,
 'isEndVersion': False,
 'articlesModificateurs': {}}

In [ ]:
data_json_2_6_3['regroupements'][2]['versions']['2023-05-21'].keys()

dict_keys(['dateDebut', 'isEndVersion', 'articlesModificateurs'])

In [ ]:
data_json_2_6_3['regroupements'][2]['versions']['2023-05-21']['articlesModificateurs'].keys()

dict_keys(['LEGIARTI000047564429'])

In [ ]:
from datetime import datetime as dt

In [ ]:
data_json_2_6_3['regroupements']

#### 2.6.3.3 Version 1 : data prep

In [ ]:
# data cleaning:

annee_recherchee = "2023"

# Vérifier la présence du champ 'regroupements' et l’analyser

for group_1 in data_json_2_6_3['regroupements']:  # Parcours regroupement d'années : list de dic

          if group_1['title'] == annee_recherchee: #filtre inject
            print(f"Année : {group_1['title']}")
                # Parcours  chaque version de texte pour l'année courante.
                #`versions` est un dictionnaire contenant les dates de chaque version.

            for version_date, version_data in group_1['versions'].items():
                # Convertir le timestamp en une date lisible
                #`version_data['dateDebut']` est en millisecondes depuis l'époque Unix (depuis le 1ᵉʳ janvier 1970 )
                # Diviser par 1000 pour obtenir les secondes, puis convertir en date lisible.

                date_debut = dt.fromtimestamp(version_data['dateDebut'] / 1000).strftime('%Y-%m-%d')

                # Affiche la date de la version (clé `version_date`) et les informations principales.
                print(f"  Version du {version_date}:")
                # Affiche la date de début de validité de la version dans un format lisible
                print(f"    Date de début : {date_debut}")
                # Affiche si cette version est la dernière pour cette date. `isEndVersion` est un booléen.
                print(f"    Est la dernière version : {'Oui' if version_data['isEndVersion'] else 'Non'}")
                # Affiche les articles modificateurs s'ils existent, ici vide `{}` dans cet exemple.
                print(f"    Articles modificateurs : {version_data['articlesModificateurs']}")




Année : 2023
  Version du 2023-05-21:
    Date de début : 2023-05-21
    Est la dernière version : Non
    Articles modificateurs : {'LEGIARTI000047564429': {'title': 'LOI n°2023-380 du 19 mai 2023 - art. 5 (V)', 'id': 'LEGIARTI000047564429', 'nature': 'CODE', 'dateDebutCible': 1684627200000, 'actions': {'MODIFICATION': {'action': 'MODIFICATION', 'parents': {'LEGISCTA000006136513': {'name': "Chapitre III : De l'examen des caractéristiques génétiques d'une personne et de l'identification d'une personne par ses empreintes génétiques", 'id': 'LEGISCTA000006136513', 'cid': 'LEGISCTA000006136513', 'cidText': 'LEGITEXT000006070721', 'nature': 'CODE', 'dateDebut': 1091836800000, 'articlesCibles': {'LEGIARTI000047569313': {'name': '16-11', 'id': 'LEGIARTI000047569313', 'cid': 'LEGIARTI000006419307', 'idParent': 'LEGISCTA000006136513', 'cidParent': 'LEGISCTA000006136513', 'cidText': 'LEGITEXT000006070721', 'sens': None, 'nature': 'CODE', 'path': 'LEGITEXT000006070721/LEGISCTA000006089697/LEGISC

#### 2.6.3.4 Version 2 : data prep

In [ ]:
from datetime import datetime as dt

# Data cleaning
annee_recherchee = "2023"

# Vérifier la présence du champ 'regroupements' , 1 ou pluisieurs valo.
for group_1 in data_json_2_6_3['regroupements']:  # Parcours regroupement d'années : list de dictionnaires
    if group_1['title'] == annee_recherchee:  # Filtre pour l'année recherchée
        print(f"Année : {group_1['title']}")

        # Parcours de chaque version de texte pour l'année courante
        for version_date, version_data in group_1['versions'].items():
          # version_date: date de version du text
          # date_debut: date de début
            # Convertir le timestamp en une date lisible mil sec
            date_debut = dt.fromtimestamp(version_data['dateDebut'] / 1000).strftime('%Y-%m-%d')

            print(f"  Version du {version_date}:")
            print(f"    Date de début : {date_debut}")

# Vérifie si dernière version
            if version_data['isEndVersion'] == True:
                print(f"    Est la dernière version : Oui")
            else:
                print(f"    Est la dernière version : Non")

            # Affiche des articles modificateurs
            print("    Articles modificateurs :")
            for article_id, article_data in version_data['articlesModificateurs'].items():
                print(f"      - ID : {article_id}")
                print(f"        Titre : {article_data['title']}")
                print(f"        Nature : {article_data['nature']}")
                date_debut_cible = dt.fromtimestamp(article_data['dateDebutCible'] / 1000).strftime('%Y-%m-%d')
                print(f"        Date de début cible : {date_debut_cible}")

# code incomplet 'parent' valo manquant
# valo anormales dates 2999 pour date de début cible pour:
# ID : LEGIARTI000046369156
# Titre : Ordonnance n°2022-1292 du 5 octobre 2022 - art. 10
# Tester

Année : 2023
  Version du 2023-05-21:
    Date de début : 2023-05-21
    Est la dernière version : Non
    Articles modificateurs :
      - ID : LEGIARTI000047564429
        Titre : LOI n°2023-380 du 19 mai 2023 - art. 5 (V)
        Nature : CODE
        Date de début cible : 2023-05-21
  Version du 2023-02-06:
    Date de début : 2023-02-06
    Est la dernière version : Non
    Articles modificateurs :
      - ID : LEGIARTI000047093354
        Titre : Décret n°2023-65 du 3 février 2023 - art. 1
        Nature : CODE
        Date de début cible : 2023-02-06
  Version du 2023-01-01:
    Date de début : 2023-01-01
    Est la dernière version : Non
    Articles modificateurs :
      - ID : LEGIARTI000044045514
        Titre : Ordonnance n°2021-1192 du 15 septembre 2021 - art. 8
        Nature : CODE
        Date de début cible : 2021-09-17
      - ID : LEGIARTI000045398819
        Titre : Ordonnance n°2022-408 du 23 mars 2022 - art. 12
        Nature : CODE
        Date de début cible : 2